# Entrenamiento y Transfer Learning con Redes Neuronales en PyTorch

Este notebook describe la implementación y entrenamiento de dos arquitecturas de redes neuronales en PyTorch: una red de capas totalmente conectadas (Fully Connected) y una red AlexNet. Incluye pasos para cargar los datos, entrenar los modelos desde cero, y realizar transfer learning adaptando los modelos para el problema de clasificación en CIFAR-100.

## Importación de Librerías

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from tqdm import tqdm

print(torch.cuda.is_available())

# En Pytorch tenemos:
# "cuda" -> GPU de Intel
# "mps" -> MacOS  Serie M (torch.backends.mps.is_available())
# "cpu" -> SIN GPU

True


En esta primera sección, importamos todas las librerías y módulos necesarios para construir, entrenar y evaluar nuestras redes neuronales en PyTorch. Aquí:

- `torch`, `torch.nn`, `torch.optim`: PyTorch y sus módulos principales para construir redes neuronales y optimización.
- `torchvision`: Módulo que permite trabajar con datasets y transformaciones de imágenes.
- `tqdm`: Herramienta para mostrar barras de progreso durante el entrenamiento del modelo.


## Definición de la Red Neuronal Fully Connected

In [2]:
class FullyConnectedNet(nn.Module):
    def __init__(self, input_size=224*224*3, hidden1=1024, hidden2=512, num_classes=1000):
        super(FullyConnectedNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

Esta sección define una red neuronal `FullyConnectedNet`, que tiene:

- Una capa de entrada que aplana la imagen (`Flatten`).
- Tres capas lineales, también llamadas Densas (`Linear`), cada una seguida por una capa de activación `ReLU` y `Dropout` para prevenir el sobreajuste.
- La última capa tiene `num_classes` neuronas, que se utiliza para la clasificación en una salida de tamaño 1000 (número de clases de ImageNet).

## Definición de la Arquitectura AlexNet

In [6]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6) # equivalente a reshape
        x = self.classifier(x)
        return x


Esta clase implementa el modelo AlexNet, una arquitectura de red neuronal convolucional (CNN) para clasificación de imágenes. Consiste en dos bloques principales:

- Bloque de características (`features`): Convoluciones y capas de pooling para extraer características.
- Bloque de clasificación (`classifier`): Compuesto de capas totalmente conectadas con Dropout para reducir el sobreajuste.

## Cargador de Datos para CIFAR-10

In [3]:
def get_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_dataset = datasets.CIFAR10(root='./CIFAR10/', train=True, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    return train_loader

Esta función prepara los datos para el entrenamiento:

- Aplica una serie de transformaciones a las imágenes (ajuste de tamaño, recorte, normalización).
- Utiliza el dataset CIFAR-10 como fuente de imágenes.
- Retorna un DataLoader para iterar por lotes de datos.

Podemos utilizar esta función porque CIFAR-10 es un dataset que ya está incluido dentro del paquete Pytorch. Si quisiéramos emplear un dataset propio, tendríamos que programar el mecanismo de carga de datos por nuestros propios medios:

1. En Pytorch, esto se logra mediante la combinación de un `torch.utils.data.Dataset` y de un `torch.utils.data.DataLoader`. Mientras que el primero se encarga de cargar en memoria los datos y leerlos, el segundo objeto se va a encargar de cargar en batches esos datos para que podamos usarlos de input en nuestras redes neuronales.
2. Los `Dataset` deben tener mínimo 3 funciones: `__init__` para inicializar la clase (esto es, definir dónde están los datos; pueden ser listas de textos, listas de *paths* a ficheros de imágenes, o una tabla), `__len__` para poder determinar el tamaño del dataset, y `__getitem__` para poder extraer del dataset el elemento de índice `idx` (esto es, poder indexar las muestras del dataset).

Por ejemplo, imaginad que nuestro dataset se compone de
- Input: Imágenes, guardadas en nuestro ordenador en la carpeta `fotos_Cancún`.
- Output: Una lista de `0` y `1` indicando si en cada foto sonreímos o no.


In [ ]:
class OurDataset(torch.utils.data.Dataset):
    """ Representa un ejemplo rapido de dataset para un problema donde las imagenes,
    guardadas en un carpeta, tienen asignada cada una una etiqueta binaria.
    """
    def __init__(self, imgs_paths: list[str], labels: list[int]):
        self.images = imgs_paths
        self.labels = labels
        assert len(imgs_paths) == len(labels), "The number of images and labels must be the same."

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

from pathlib import Path

# imaginamos nuestra carpeta, extraemos los ficheros de las fotos
photos = list(Path("./fotos_Cancún/").glob("*.png"))
# las etiquetas imaginamos que las hemos cargado de algún fichero o tabla
labels = [0, 1, 0, 1, 1, ...]

cancun_dataset = OurDataset(photos, labels)
cancun_dataloader = DataLoader(cancun_dataset, batch_size=8, shuffle=False)
# si hacemos cancun_dataset[0],
# >>> "./fotos_Cancun/photo_0.png", 0

Esto no sería suficiente, por el siguiente detalle: Daos cuenta de que al extraer el elemento `idx` en la función `__getitem__`, no estamos realmente cargando la imagen, sino el path a la imagen. Para verdaderamente cargar una imagen (además de poder incluir cualquier tipo de preprocesamiento *online*) debemos definir una función adicional. Por tanto,


In [ ]:
import cv2

def load_photo(path):
  # podemos complicar esta funcion todo lo que deseemos
  return cv2.imread(path).cvtColor(cv2.COLOR_BGR2RGB)


class OurDataset(torch.utils.data.Dataset):
    """ Representa un ejemplo rapido de dataset para un problema donde las imagenes,
    guardadas en un carpeta, tienen asignada cada una una etiqueta binaria.
    """
    def __init__(self, imgs_paths: list[str], labels: list[int]):
        self.paths = imgs_paths
        self.labels = labels
        assert len(imgs_paths) == len(labels), "The number of images and labels must be the same."

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return load_photo(self.paths[idx]), self.labels[idx]

from pathlib import Path

# imaginamos nuestra carpeta, extraemos los ficheros de las fotos
photos = list(Path("./fotos_Cancún/").glob("*.png"))
# las etiquetas imaginamos que las hemos cargado de algún fichero o tabla
labels = [0, 1, 0, 1, 1, ...]

cancun_dataset = OurDataset(photos, labels)

# si hacemos ahora cancun_dataset[0]
# >>> np.ndarray de imagen, 0

Para pasar de este dataset donde podemos extraer muestras una a una a un procesador de carga de imágenes por batches de manera automática,

In [ ]:
loader = DataLoader(cancun_dataset, batch_size=8, shuffle=True)

Los `DataLoader` funcionan como un iterador en Python, de tal manera que no podemos hacer `loader[0]`, pero sí podemos iterar, obteniendo tensores del tamaño de nuestro batch size.

**Recordad: Necesitaremos usar Dataset & DataLoader cuando implementemos sistemas con nuestros propios datos, es importante familiarizarse con ellos cuanto antes.**

## Selección del Modelo

In [8]:
def select_model(model_name):
    if model_name == 'fc_net':
        model = FullyConnectedNet()
    elif model_name == 'alexnet':
        model = AlexNet()
    else:
        raise ValueError("Invalid model name! Choose 'fc_net' or 'alexnet'.")
    return model

demo_model = select_model("alexnet")
print(demo_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 

In [10]:
demo_layer = demo_model.features
print(demo_layer)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)


## Función de Entrenamiento

In [11]:
def train_model(model, train_loader, device, epochs=10, lr=0.001):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")


En la función anterior, fijaos en las pocas diferencias con respecto a Keras:
- Diferentes nombres, pero similares.
- Tenemos que indicar al modelo que va a tener que calcular gradientes (`model.train()`).
- Generación de los batches por nosotros mismos mediante DataLoader.
- Mover el modelo, así como los datos, al `device` que usemos.
- En cada batch, olvidar el gradiente calculado anteriormente (optimizer.zero_grad()).
- Backpropagation indicado como `loss.backward()`.
- Siguiente paso de aprendizaje mediante `optimizer.step()`.

Puede parecer un engorro comparado con Keras, pero ahora que sabemos qué significan estos conceptos, podemos tener control sobre ellos. Por ejemplo, podemos querer guardar el estado de un modelo en un instante concreto, junto con el estado de su optimizador para continuar en otro momento o en otro día con el entrenamiento (https://pytorch.org/tutorials/beginner/saving_loading_models.html).

## Configuración y Entrenamiento del Modelo

In [12]:
BS = 256
MODEL = "alexnet"
EPOCHS = 2
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = get_dataloader(BS)
model = select_model(MODEL)
train_model(model, train_loader, device, EPOCHS, LR)


100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]
100%|██████████| 196/196 [02:32<00:00,  1.29it/s]


Epoch 1, Loss: 2.155084535783651


100%|██████████| 196/196 [02:13<00:00,  1.46it/s]

Epoch 2, Loss: 1.4690938154045416


## Transfer Learning

In [13]:
def modify_model_for_cifar100(model, model_name):
    if model_name == 'fc_net':
        model.fc3 = nn.Linear(model.fc2.out_features, 100)
    elif model_name == 'alexnet':
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 100)
    else:
        raise ValueError("Invalid model name! Choose 'fc_net' or 'alexnet'.")
    return model


## Transfer Learning y Evaluación

In [14]:
def get_cifar100_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize(224),          # Resize to 224x224 for compatibility
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    cifar100_train = datasets.CIFAR100(root='./CIFAR100/', train=True, transform=transform, download=True)
    cifar100_test = datasets.CIFAR10(root='./CIFAR100/', train=False, transform=transform, download=True)

    train_loader = DataLoader(cifar100_train, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

def train_transfer_learning(model, train_loader, test_loader, device, epochs=10, lr=0.001):
    model = model.to(device)

    # Freeze all parameters except the last layer
    for param in model.parameters():
        param.requires_grad = False

    # Only the modified layer's parameters will require gradients
    if hasattr(model, 'fc3'):
        model.fc3.weight.requires_grad = True
    else:
        model.classifier[-1].weight.requires_grad = True

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [15]:
model = modify_model_for_cifar100(model, MODEL)
train_loader, test_loader = get_cifar100_dataloader(batch_size=64)
train_transfer_learning(model, train_loader, test_loader, device, epochs=EPOCHS, lr=LR)


100%|██████████| 169M/169M [00:19<00:00, 8.70MB/s]
100%|██████████| 170M/170M [00:13<00:00, 12.3MB/s]
100%|██████████| 782/782 [01:42<00:00,  7.63it/s]


Epoch 1, Loss: 4.204865190684033


100%|██████████| 782/782 [01:34<00:00,  8.24it/s]


Epoch 2, Loss: 3.987745090213883
Test Accuracy: 0.96%
